In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import urllib.request
import sys

In [ ]:
def getCompanylist(url):
    try:
        html = requests.get(url, headers=headers,timeout=10)
    except:    
        print('fail1')
        try:
            html = requests.get(url, headers=headers,timeout=10)
        except:
            print('fail2')
            try:
                html = requests.get(url, headers=headers,timeout=10)
            except:
                print('fail3')
    soup = BeautifulSoup(html.text, 'lxml')
    
    companyName = []
    companyNumber = []
    status = []
    link = []
    
    
    tableContent = soup.find_all("td", {"class": "govuk-table__cell"})
    

    nextpage=soup.find_all('div', {'class': "pagination"})
    nexpageUrl = nextpage[0].find('a',{"id":"nextLink"})
    urltext= "https://find-and-update.company-information.service.gov.uk/alphabetical-search/"+str( nexpageUrl.get('href'))
    print(urltext)

    for i in range(int(len(tableContent)/3)):
        indexNumber = i*3
        companyName.append(tableContent[indexNumber].find(text=True)) 
        number = tableContent[indexNumber+1].find(text=True)
        companyNumber.append(number)
        status.append(tableContent[indexNumber+2].find(text=True))
        link.append("https://find-and-update.company-information.service.gov.uk/company/"+str(number)) 

    sub_df = pd.DataFrame({'companyName': companyName, 'companyNumber': companyNumber,'status': status,'link': link })
    
    return sub_df,urltext

In [2]:
df_list = pd.read_csv('list.csv',encoding='latin1')
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}


In [64]:



df_list
    


,Unnamed: 0,companyName,companyNumber,status,link
0,0,\ COMPANY LTD,5060411,Active,https://find-and-update.company-information.se...
1,1,* LTD,5062633,Active,https://find-and-update.company-information.se...
2,2,? LTD,5063820,Dissolved,https://find-and-update.company-information.se...
3,3,*+ LTD,5153997,Active,https://find-and-update.company-information.se...
4,4,.CO LTD,8013348,Active,https://find-and-update.company-information.se...
...,...,...,...,...,...
3995,35,A.A.C. SERVICES LIMITED,2433281,Active,https://find-and-update.company-information.se...
3996,36,AAC (SERVICES) SOUTHERN LIMITED,8649344,Active,https://find-and-update.company-information.se...
3997,37,AAC SERVICE (SUSSEX) LIMITED,7957625,Active,https://find-and-update.company-information.se...
3998,38,AAC SHIPPING LIMITED,12496624,Dissolved,https://find-and-update.company-information.se...


In [53]:
content = soup.find_all("div", attrs = {"id": "content-container"})


'Private limited Company'

In [61]:

industryCode = content[0].find("span",{"id":"sic0"}).find(text= True).strip()
industryCode

'47910 - Retail sale via mail order houses or via Internet'

In [68]:
def getDetail(url):
    try:
        html = requests.get(url, headers=headers,timeout=10)
    except:    
        print('fail1')
        try:
            html = requests.get(url, headers=headers,timeout=10)
        except:
            print('fail2')
            try:
                html = requests.get(url, headers=headers,timeout=10)
            except:
                print('fail3')
    soup = BeautifulSoup(html.text, 'lxml')
    address= ''
    companyType = ''
    incorporated = ''
    industryCode = ''
    
    content = soup.find_all("div", attrs = {"id": "content-container"})
    
    address = content[0].find("dl").find_all(text= True)[3].strip()
    
    companyTypelevel = content[0].find_all("dl",{"class":"column-two-thirds"})
    companyType = companyTypelevel[1].find('dd').find(text= True).strip()
    
    incorporated  = content[0].find("dd",{"id":"company-creation-date"}).find(text= True).strip()
    
    industryCode = content[0].find("span",{"id":"sic0"}).find(text= True).strip()
    
    keys = ['address','companyType','incorporated','industryCode']
    values = [address,companyType,incorporated,industryCode]
    dictionary = dict(zip(keys, values))
    sub_df=pd.DataFrame(dictionary,index = [0])

    return  sub_df

    

In [71]:
len(df_detail)

0

In [ ]:
df_detail = pd.DataFrame()
for i in range(len(df_list)):
    try:
        df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
    except:
        df_detail = df_detail.append(pd.Series(),ignore_index=True) 
df_detail.to_csv('df_detail.csv')        




C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.con

C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.con

C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.con

C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.con

C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.con

...

C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.con

C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.con

C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.con

C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.con

C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_detail = df_detail.append(getDetail(df_list['link'].iloc[i]))
C:\Users\SHIHAO~1\AppData\Local\Temp/ipykernel_19212/1018290731.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.con

In [ ]:
def getPeopleDetail(url):
    try:
        html = requests.get(url, headers=headers,timeout=10)
    except:    
        print('fail1')
        try:
            html = requests.get(url, headers=headers,timeout=10)
        except:
            print('fail2')
            try:
                html = requests.get(url, headers=headers,timeout=10)
            except:
                print('fail3')
    soup = BeautifulSoup(html.text, 'lxml')
    address= ''
    companyType = ''
    companyStatus = ''
    incorporated = ''
    industryCode = ''
    